In [8]:
import pandas as pd
import numpy as np
import psycopg2


In [2]:
### reading the dataset
customers = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
products_translation = pd.read_csv('product_category_name_translation.csv')

### data quality check

### checking for 
















In [3]:
def show_empty_columns_percentages(df):
    empty_percentages = df.isnull().mean() * 100
    return empty_percentages[empty_percentages > 0]

In [4]:
show_empty_columns_percentages(customers)

Series([], dtype: float64)

In [5]:
show_empty_columns_percentages(geolocation)

Series([], dtype: float64)

In [11]:
# ----------------------------------------
# 1. Your settings — edit these
# ----------------------------------------
CSV_FILE = "olist_customers_dataset.csv"        # path to your CSV file
TABLE_NAME = "customers"        # table name in PostgreSQL

DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "postgres"
DB_USER = "myuser"
DB_PASS = "mypassword"

# ----------------------------------------
# 2. Read the CSV
# ----------------------------------------
df = pd.read_csv(CSV_FILE)

print(f"CSV loaded. Rows: {len(df)}, Columns: {list(df.columns)}")

# ----------------------------------------
# 3. Connect to PostgreSQL
# ----------------------------------------
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)
cursor = conn.cursor()

print("Connected to database successfully.")

# ----------------------------------------
# 4. Create the table automatically from CSV columns
# ----------------------------------------
# This maps pandas dtypes to PostgreSQL types
def get_pg_type(dtype):
    if "int" in str(dtype):
        return "INTEGER"
    elif "float" in str(dtype):
        return "FLOAT"
    elif "datetime" in str(dtype):
        return "TIMESTAMP"
    else:
        return "TEXT"

columns = ", ".join([
    f"{col} {get_pg_type(df[col].dtype)}"
    for col in df.columns
])

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        {columns}
    );
"""

cursor.execute(create_table_query)
conn.commit()

print(f"Table '{TABLE_NAME}' created (or already exists).")

# ----------------------------------------
# 5. Insert rows into the table
# ----------------------------------------
inserted = 0

for _, row in df.iterrows():
    placeholders = ", ".join(["%s"] * len(row))
    col_names = ", ".join(df.columns)
    insert_query = f"INSERT INTO {TABLE_NAME} ({col_names}) VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))
    inserted += 1

conn.commit()
print(f"Done! {inserted} rows inserted into '{TABLE_NAME}'.")

# ----------------------------------------
# 6. Close connection
# ----------------------------------------
cursor.close()
conn.close()

CSV loaded. Rows: 99441, Columns: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
Connected to database successfully.
Table 'customers' created (or already exists).
Done! 99441 rows inserted into 'customers'.


SyntaxError: invalid syntax (357091671.py, line 1)